# How to use paramManager
paramManager writes and reads parameter files for data file sets, one structured parameter dictionary per data file.
The routines here are just for demonstrative purposes, feel free to use any library/tools to extract the parameter values that you want. Then use paramManager to create parameter files with those values.

In [ ]:
import numpy as np
import librosa # conda install -c conda-forge librosa
import librosa.display

import os  # for mkdir
from os import listdir
from os.path import isfile, join
import sys

%matplotlib inline
import matplotlib.pyplot as plt

##-----------   paramManager  interface   ------------------##
from paramManager import paramManager

## paramManager on a directory (normal operation)

In [ ]:
datapath='data'  #root folder
parampath='data' 
# print the files (but not the directories) in datapath
fileList = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(datapath, f))]
#fileList,fnameList = listDirectory_all(datapath)
len(fileList)

In [ ]:
# paramManager, create 
pm=paramManager(datapath, parampath)  ##-----------   paramManager  interface ------------------##

In [ ]:
# initialize parameter files for the data (only necessary if they don't already exist)
# set overwrite=True for initial creation
# The boolean parameter tells whether to overwrite current parameter directory (default is False)
pm.initParamFiles(overwrite=True) ##-----------   paramManager  interface ------------------##

In [ ]:
# paramManager , check that data files and paramfiles match in number and name
pm.checkIntegrity() ##-----------   paramManager  interface   ------------------##

In [ ]:
# param Manager , get the parameter dictionary for a file 
# This is the critical function for getting parameters for training nets. 

##-----------   paramManager  interface   ------------------##
foo=pm.getParams(datapath+'/brass_acoustic_018-070-127') #extension is optional
foo

## paramManager on a single file (special use case)

In [ ]:
if 0 : # this is how do run paramManager on a single file. The interface needs improvement!

    datapath=datapath+'/brass_acoustic_018-070-127.wav'  #datapath is pointing to an audio file
    parampath='data'           #parampath is the parent folder containing/will contain the corresponding param file 
    pm=paramManager(datapath, parampath)

    pm.initParamFiles(overwrite=False) #set overwrite=True if creating paramfile for the first time 
    pm.checkIntegrity() 

    foo=pm.getParamNames(datapath) #extension is optional
    foo

# First a little essentia demo

In [ ]:
# http://essentia.upf.edu/documentation/documentation.html
import essentia # conda install -c thomasfillon essentia
import essentia.standard


In [ ]:
testfile = datapath + '/' + 'reed_acoustic_000-072-127.wav'

loader = essentia.standard.AudioLoader(filename=testfile) #Monoloader doesn't get metadata

# and then we actually perform the loading:
(audio, sr,chans,_,_,_) = loader()
print("shape of audio is  is {}".format(audio.shape))
print("chans is {}".format(chans))
print("audio is {}".format(audio))
audio=audio[:,0] #get (the) one (and only) channel

import IPython
IPython.display.Audio(testfile)


In [ ]:
PitchMelodia = essentia.standard.PitchMelodia( binResolution=2, sampleRate=sr, guessUnvoiced=True) # donothing??: , minFrequency=10, maxFrequency=200)
p, pc = PitchMelodia(audio )
pitch_times = np.linspace(0.0,len(audio)/sr,len(p) )
plt.figure()
plt.title("Pitch:")
plt.plot(pitch_times,p)

# paramManager - analyze data files to create a new parameter with Essentia

In [ ]:
# Get a list of data file names so we can pass them to essentia for analysis
for fname in pm.getFullPathNames(datapath) :
    print("processing {}".format(fname))
    loader = essentia.standard.AudioLoader(filename=fname) #Monoloader doesn't get metadata
    (audio, sr,chans,_,_,_) = loader()
    audio=audio[:,0] #get (the) one (and only) channel
    
    PitchMelodia = essentia.standard.PitchMelodia( binResolution=2, sampleRate=sr, guessUnvoiced=True) # donothing??: , minFrequency=10, maxFrequency=200)
    p, pc = PitchMelodia(audio )
    
    # Need time points for the parameter file, which essentia does not provide
    pitch_times = np.linspace(0.0,len(audio)/sr,len(p) )
    if (not (len(p) == len(pitch_times))) :
        print("Warning, num parameters not equal to time points")
    
    # Now add the data to the parameter file
    ##-----------   paramManager  interface   ------------------##
    pm.addParam(fname, "pitch", pitch_times, p, units="frequency", nvals=0, minval=0, maxval=1)
    
    

In [ ]:
len(audio)/sr

In [ ]:
# Here we show how to get a particular parameater from the pdict for a datafile
##-----------   paramManager  interface   ------------------##
fooparams=pm.getParams(datapath + '/' + 'reed_acoustic_000-072-127.wav')
foopitch=fooparams['pitch']

#now just show that it looks the same as the essential data 
title=fooparams['meta']['filename']
plt.figure()
plt.title(title)
plt.plot(foopitch['times'], foopitch['values'])

# Librosa demo

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

sig,sr=librosa.core.load(datapath + "/" + "reed_acoustic_000-072-127.wav", sr=None)
rmse=librosa.feature.rms(y=sig)[0]
rmse_times = np.linspace(0.0,len(sig)/sr, len(rmse) )

plt.figure()
plt.subplot(2, 1, 1)
#plt.semilogy(rmse.T, label='RMS Energy')
plt.plot(rmse_times, rmse, label='RMS Energy')
plt.legend(loc='best')

plt.subplot(2, 1, 2)
librosa.display.waveplot(sig, sr=16000)

print('duration=',len(sig)/sr)

In [ ]:
datafiles = [fname for fname in pm.getFullPathNames(datapath)]
#check all durations are the same
def check_duration(filelist):
    """use librosa's get_duration method to find the duration of all files in filelist"""
    filedurations = [librosa.core.get_duration(filename=f) for f in filelist]
    assert filedurations.count(filedurations[0]) == len(filedurations), "File durations are not all the same!"
    return filedurations[0]

duration = check_duration(datafiles) 
print(duration)

# paramManager - analyze data files to create a new parameter with LibRosa 

In [ ]:
# parameters: rmse - frame_length=4000, hop_length=1000
#             spectral centroid - n_fft=4000, hop_length=1000 


for fname in pm.getFullPathNames(datapath) :
    print("processing {}".format(fname))

    sig,sr=librosa.core.load(fname, sr=None)
    rmse=librosa.feature.rms(y=sig, frame_length=4000, hop_length=1000)[0]
    sc = librosa.feature.spectral_centroid(y=sig, sr=16000, n_fft=4000, hop_length=1000)[0]
    
    sig_times = np.linspace(0.0,len(sig)/sr, len(sig))
    rmse_times = np.linspace(0.0,len(sig)/sr, len(rmse))
    sc_times = np.linspace(0.0,len(sig)/sr, len(sc))
    
    """
    fig, ax1 = plt.subplots(figsize=(16,3))

    lns1=ax1.plot(sig_times,sig,label='sig',color='k')
    lns2=ax1.plot(rmse_times,rmse,label='rmse')
    
    ax2 = ax1.twinx()
    lns3=ax2.plot(sc_times,sc,label='sc',color='r')
    
    lns = lns1+lns2+lns3
    labs = [l.get_label() for l in lns]
    ax1.legend(lns, labs, loc=0)
    ax1.set_ylim(-1, 1)
    ax2.set_ylim(0,8000)
    plt.grid()
    plt.show()
    """
    break
print(len(sig))
print(len(rmse))
print(len(sc))
    
    # Now add the data to the parameter file
    ##-----------   paramManager  interface   ------------------##
    #pm.addParam(fname, "rmse", rmse_times, rmse, units="energy", nvals=0, minval=0, maxval=None)
    #pm.addParam(fname, "spec_centroid", sc_times, sc, units="frequency", nvals=0, minval=0, maxval=None)
    

In [ ]:
#parameters frame 30ms with 50% hop sr~67
# parameters: rmse - frame_length=480, hop_length=240
#             spectral centroid - n_fft=480, hop_length=240
#             mfcc - n_mfcc=13, dct_type=2, lifter=0, n_fft=480, hop_length=240 
mfcc_maxs = np.zeros((38,13))
mfcc_mins = np.zeros((38,13))
i = 0

for fname in pm.getFullPathNames(datapath) :
    print("processing {}".format(fname))

    sig,sr=librosa.core.load(fname, sr=None)
    rmse=librosa.feature.rmse(y=sig, frame_length=480, hop_length=240)[0]
    sc = librosa.feature.spectral_centroid(y=sig, sr=16000, n_fft=480, hop_length=240)[0]
    mfcc = librosa.feature.mfcc(y=sig, sr=16000, n_mfcc=13, n_fft=480, hop_length=240)
    
    sig_times = np.linspace(0.0,len(sig)/sr, len(sig))
    rmse_times = np.linspace(0.0,len(sig)/sr, len(rmse))
    sc_times = np.linspace(0.0,len(sig)/sr, len(sc))
    
    #sc_log = np.log(sc)
    sc_logone = np.log1p(sc)
    sc_norm = sc_logone / np.log1p(8000)
    #plt.figure(figsize=(10, 4))
    #plt.plot(sc,label='sc')
    #plt.plot(sc_logone,label='log(1+sc)')
    #plt.plot(sc_log,label='log(sc)')
    #plt.plot(sc_norm,label='sc norm')
    #plt.ylim((0, 1))
    #plt.legend()
    #plt.grid()
    #plt.show()
    
    
    fig, ax1 = plt.subplots(figsize=(16,3))

    lns1=ax1.plot(sig_times,sig,label='sig',color='k')
   
    ax2 = ax1.twinx()
    lns2=ax2.plot(rmse_times,rmse,label='rmse')
    lns3=ax2.plot(sc_times,sc_norm,label='sc',color='r')
    
    lns = lns1+lns2+lns3
    labs = [l.get_label() for l in lns]
    ax1.legend(lns, labs, loc=0)
    ax1.set_ylim(-1, 1)
    ax2.set_ylim(0,1)
    plt.grid()
    plt.show()
    
    """
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mfcc, x_axis='time', sr=16000, hop_length=240)
    plt.colorbar()
    plt.title('MFCC')
    plt.tight_layout()
    plt.show()
    """
    mfcc_max = np.amax(np.absolute(mfcc),axis=1)
    mfcc_min = np.amin(np.absolute(mfcc),axis=1)
    mfcc_maxs[i] = mfcc_max
    mfcc_mins[i] = mfcc_min
    #print(max(x.min(), x.max(), key=abs))
    
    
    i += 1
    
    
    # Now add the data to the parameter file
    ##-----------   paramManager  interface   ------------------##
    pm.addParam(fname, "rmse", rmse_times, rmse, units="energy", nvals=0, minval=0, maxval=None)
    pm.addParam(fname, "spec_centroid", sc_times, sc_norm, units="frequency", nvals=0, minval=0, maxval=None)

print(sig.shape)
print(rmse.shape)
#print(sc.max(),sc.min())
#print(sc)
#print(sc_log)

max_mfcc = np.amax(mfcc_maxs,axis=0)
min_mfcc= np.amin(mfcc_mins,axis=0)
print(max_mfcc)
print(min_mfcc)

In [ ]:
# paramManager - analyze data files to create a new parameter with LibRosa 

In [ ]:
# parameters: rmse - frame_length=4000, hop_length=1000
#             spectral centroid - n_fft=4000, hop_length=1000 


for fname in pm.getFullPathNames(datapath) :
    print("processing {}".format(fname))

    sig,sr=librosa.core.load(fname, sr=None)
    rmse=librosa.feature.rms(y=sig, frame_length=4000, hop_length=1000)[0]
    sc = librosa.feature.spectral_centroid(y=sig, sr=16000, n_fft=4000, hop_length=1000)[0]
    
    sig_times = np.linspace(0.0,len(sig)/sr, len(sig))
    rmse_times = np.linspace(0.0,len(sig)/sr, len(rmse))
    sc_times = np.linspace(0.0,len(sig)/sr, len(sc))
    
    """
    fig, ax1 = plt.subplots(figsize=(16,3))

    lns1=ax1.plot(sig_times,sig,label='sig',color='k')
    lns2=ax1.plot(rmse_times,rmse,label='rmse')
    
    ax2 = ax1.twinx()
    lns3=ax2.plot(sc_times,sc,label='sc',color='r')
    
    lns = lns1+lns2+lns3
    labs = [l.get_label() for l in lns]
    ax1.legend(lns, labs, loc=0)
    ax1.set_ylim(-1, 1)
    ax2.set_ylim(0,8000)
    plt.grid()
    plt.show()
    """
    break
print(len(sig))
print(len(rmse))
print(len(sc))
    
    # Now add the data to the parameter file
    ##-----------   paramManager  interface   ------------------##
    #pm.addParam(fname, "rmse", rmse_times, rmse, units="energy", nvals=0, minval=0, maxval=None)
    #pm.addParam(fname, "spec_centroid", sc_times, sc, units="frequency", nvals=0, minval=0, maxval=None)
    

In [ ]:
#parameters frame 30ms with 50% hop sr~67
# parameters: rmse - frame_length=480, hop_length=240
#             spectral centroid - n_fft=480, hop_length=240
#             mfcc - n_mfcc=13, dct_type=2, lifter=0, n_fft=480, hop_length=240 
mfcc_maxs = np.zeros((38,13))
mfcc_mins = np.zeros((38,13))
i = 0

for fname in pm.getFullPathNames(datapath) :
    print("processing {}".format(fname))

    sig,sr=librosa.core.load(fname, sr=None)
    rmse=librosa.feature.rms(y=sig, frame_length=480, hop_length=240)[0]
    sc = librosa.feature.spectral_centroid(y=sig, sr=16000, n_fft=480, hop_length=240)[0]
    mfcc = librosa.feature.mfcc(y=sig, sr=16000, n_mfcc=13, n_fft=480, hop_length=240)
    
    sig_times = np.linspace(0.0,len(sig)/sr, len(sig))
    rmse_times = np.linspace(0.0,len(sig)/sr, len(rmse))
    sc_times = np.linspace(0.0,len(sig)/sr, len(sc))
    
    #sc_log = np.log(sc)
    sc_logone = np.log1p(sc)
    sc_norm = sc_logone / np.log1p(8000)
    #plt.figure(figsize=(10, 4))
    #plt.plot(sc,label='sc')
    #plt.plot(sc_logone,label='log(1+sc)')
    #plt.plot(sc_log,label='log(sc)')
    #plt.plot(sc_norm,label='sc norm')
    #plt.ylim((0, 1))
    #plt.legend()
    #plt.grid()
    #plt.show()
    
    
    fig, ax1 = plt.subplots(figsize=(16,3))

    lns1=ax1.plot(sig_times,sig,label='sig',color='k')
   
    ax2 = ax1.twinx()
    lns2=ax2.plot(rmse_times,rmse,label='rmse')
    lns3=ax2.plot(sc_times,sc_norm,label='sc',color='r')
    
    lns = lns1+lns2+lns3
    labs = [l.get_label() for l in lns]
    ax1.legend(lns, labs, loc=0)
    ax1.set_ylim(-1, 1)
    ax2.set_ylim(0,1)
    plt.grid()
    plt.show()
    
    """
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mfcc, x_axis='time', sr=16000, hop_length=240)
    plt.colorbar()
    plt.title('MFCC')
    plt.tight_layout()
    plt.show()
    """
    mfcc_max = np.amax(np.absolute(mfcc),axis=1)
    mfcc_min = np.amin(np.absolute(mfcc),axis=1)
    mfcc_maxs[i] = mfcc_max
    mfcc_mins[i] = mfcc_min
    #print(max(x.min(), x.max(), key=abs))
    
    
    i += 1
    
    
    # Now add the data to the parameter file
    ##-----------   paramManager  interface   ------------------##
    pm.addParam(fname, "rmse", rmse_times, rmse, units="energy", nvals=0, minval=0, maxval=None)
    pm.addParam(fname, "spec_centroid", sc_times, sc_norm, units="frequency", nvals=0, minval=0, maxval=None)

print(sig.shape)
print(rmse.shape)
#print(sc.max(),sc.min())
#print(sc)
#print(sc_log)

max_mfcc = np.amax(mfcc_maxs,axis=0)
min_mfcc= np.amin(mfcc_mins,axis=0)
print(max_mfcc)
print(min_mfcc)

In [ ]:
mfcc_times = np.linspace(0.0,len(sig)/sr, mfcc.shape[1])
print(mfcc_times.shape)

for fname in pm.getFullPathNames(datapath) :
    print("processing {}".format(fname))
    
    mfcc = librosa.feature.mfcc(y=sig, sr=16000, n_mfcc=13, n_fft=480, hop_length=240)
    #print(mfcc)
    for i in range(mfcc.shape[0]):
        mfcc[i,:] = (mfcc[i,:] - min_mfcc[i]) / (max_mfcc[i] - min_mfcc[i])
        
        pm.addParam(fname, "mfcc"+str(i), mfcc_times, mfcc[i,:], units="None", nvals=0, minval=0, maxval=None)

    #print(mfcc)
    

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mfcc, x_axis='time', sr=16000, hop_length=240)
    plt.colorbar()
    plt.title('MFCC')
    plt.tight_layout()
    plt.show()

    

# paramManager, add params from info stored in filenames

In [ ]:
import re

for fname in pm.getFullPathNames(datapath) :
    print("processing {}".format(fname))
    
    # first, instrument ID
    if fname.find("brass") >=0 : 
        instID=0
    else : 
        instID=1
    print("    {} is in category {}".format(fname, 1))
            
    ##-----------   paramManager  interface   ------------------##
    pm.addParam(fname, "instID", [0.0,duration], [instID,instID], units="category", nvals=2, minval=0, maxval=1)
    

    #now midiPitch
    n=re.findall(r'(?<=-).*?(?=-)', fname)[0] #3-digit pitch number wedged between dashes (-) in name
    if (n[0]=='0') :
        midinum=int(n[1:])
    else :
        midinum=int(n)
    print("    midi number is {}".format(midinum))
    
    ##-----------   paramManager  interface   ------------------##
    pm.addParam(fname, "midiPitch", [0.0,duration], [midinum,midinum], units="midiPitch", nvals=128, minval=0, maxval=127)
    

In [ ]:
# Now lets have a look at one of our paramfiles
foo=pm.getParams(datapath + '/' + 'reed_acoustic_000-072-127.wav') #the path and extension are optional

# paramManager, add params from info stored in pandas dataframe

In [ ]:
if 0   : #
    import pandas as pd

    dvectors = pd.read_pickle('/enrollments/full_3layer.pkl')
    dvectors.head()

In [ ]:
if 0   : #
    from utils.myUtils import extract_nsynth_instrument,extract_nsynth_pitch
    from os.path import basename

    columns = [str(i) for i in range(16)]

    for fname in pm.getFullPathNames(datapath) :
        #print(fname)
        base = basename(fname)
        print("processing {}".format(base))
        inst = extract_nsynth_instrument(base)
        pitch = extract_nsynth_pitch(base)
        a = dvectors.loc[lambda dvectors: (dvectors['inst'] == inst) & (dvectors['pitch'] == pitch)] 
        a_flat = np.squeeze(a[columns].values)
        #print(len(a_flat))

        for i in range(len(a_flat)):
            pm.addParam(fname, "dvector_"+str(i), [0.0,duration], [a_flat[i],a_flat[i]], units="category", nvals=0, minval='null', maxval='null')

# Resampling (interpolation) demo

In [ ]:
#sometimes we need parameter values at a different sampling rate
#for example we have a section of data made up of 6 samples, taken from 0.0s to 0.03s of reed_acoustic_000-072-127.wav
#but there are only 2 rmse (parameter) values associated with this duration
#to get a rmse (parameter) for each of our 6 samples, load the parameter file using getParams then pass it to resampleParam as follows:

pm.resampleParam(foo,"rmse",6,0.0,0.03,verbose=True)

#this function uses scipy's linear interpolation method

In [ ]:
#the default behaviour of resampleParam is not to overwrite the original param file

foo=pm.getParams(datapath + '/' + 'reed_acoustic_000-072-127.wav')
foo

In [ ]:
#but this can be changed if overwrite=True

pm.resampleParam(foo, "rmse",10,0.15,0.25,verbose=True,overwrite=True)
foo=pm.getParams(datapath + '/' + 'reed_acoustic_000-072-127.wav')
foo

In [ ]:
#instead of resampling 1 parameter at a time, use resampleAllParams to automatically interpolate all of them (will ignore meta)

paramdict = pm.resampleAllParams(foo,5,1.1,1.2,verbose=True)

In [ ]:
#we can even select which parameters to resample using the prop variable 
paramdict = pm.resampleAllParams(foo,5,1.1,1.2,prop=['instID','midiPitch'],verbose=True)

In [ ]:
foo=pm.getParams(datapath + '/' + 'reed_acoustic_000-070-127.params')
foo

In [ ]:
if 0 :#
    pm.resampleParam(foo,"dvector_0",4,0.0,0.5,verbose=True)